In [1]:
import matplotlib.pyplot as plt
import pickle
import os
import snap
from itertools import combinations
import numpy as np
from google.cloud import bigquery
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx

In [2]:
with open('./data/project_authors_2016_q1_k25core.pickle', 'rb') as picklefile:
    project_authors_q1 = pickle.load(picklefile)

In [34]:
dimensions=32
window=10
iterations=10
p=10.
q=0.1
num_walks=25
walk_length = 400

In [4]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(project_authors_q1['author_id'].unique())
nx_weighted.add_nodes_from(project_authors_q1['project_id'].unique())
for _, row in project_authors_q1.iterrows():
    nx_weighted.add_edge(-row['project_id'], row['author_id'], weight=row['commit_times'])

In [5]:
len(nx_weighted)

7376

In [14]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 3min 4s, sys: 1.47 s, total: 3min 6s
Wall time: 3min 5s


In [35]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25
9 / 25
10 / 25
11 / 25
12 / 25
13 / 25
14 / 25
15 / 25
16 / 25
17 / 25
18 / 25
19 / 25
20 / 25
21 / 25
22 / 25
23 / 25
24 / 25
25 / 25
CPU times: user 24min 35s, sys: 26.4 s, total: 25min 2s
Wall time: 25min
CPU times: user 48min 14s, sys: 14 s, total: 48min 28s
Wall time: 16min 18s


In [28]:
len(topk)

25322

In [26]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)
with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)

In [36]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(3508, 0.1300656260427867)

100, 25 (2864, 0.11310323039254404)
100, 100 (3459, 0.1282488598865448)

In [23]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(3159, 0.12475317905378722)

In [20]:
len(project_authors_q1['author_id'].unique())

3053

In [24]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))
with open('./data/node2vec-weighted-p10-q01-100-100.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)